## Visualizations

In [109]:
%useLatestDescriptors
%use dataframe
%use kandy

In [76]:
fun DataFrame.Companion.readMetricsJson(json: Path) = readJson(json.pathString).cast<Int>()

val resultDir = Path("results/instances")
val metricsPath = Path("seru_result/metrics.json")
val instances = resultDir.toFile()
    .walk()
    .filter { it.isDirectory && (it.name.startsWith("f") || it.name.startsWith("v")) }
    .map { it.toPath() }


In [123]:
val instancesDfs = instances
    .take(4)
    .map {
        it.listDirectoryEntries()
            .map { DataFrame.readMetricsJson(it / metricsPath) }
            .reduce { acc, dataFrame -> acc.concat(dataFrame) }
    }
    .reduce { acc, df -> acc.concat(df) }

In [135]:
val groupedInstanceDfs = instancesDfs
    .groupBy { InputDir.map { it.removePrefix("study/instances/") } }
    .sortBy { InputDir.map { it.substringBefore("/") } }

In [136]:
val meanDf = groupedInstanceDfs.mean()

meanDf
    .plot {
        bars {
            x(InputDir) {
                axis {
                    name = "Instance"
                }
            }
            y("TotalTimeMillis")
        }

        layout {
            style {
                xAxis.text {
                    angle = 45.0
                }
            }
        }
    }

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="y6Bdoj"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"TotalTimeMillis":[22104.666666666668,14047.666666666666,10345.333333333334,1666.0],
"InputDir":["extra/issue_2_inlined/final","panic/issue_2584/v1","panic/issue_2584/v2","panic/issue_2584/final"]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Instance"
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"InputDir",
"y":"TotalTimeMillis"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}],
"theme":{
"axis_text_x":{
"angle":45.0,
"blank":false
},
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false
}
};
 var plotContainer = document.getElementById("y6Bdoj");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 extra/issue_2_inlined/final 
 
 
 
 
 
 
 
 
 panic/issue_2584/v1 
 
 
 
 
 
 
 
 
 panic/issue_2584/v2 
 
 
 
 
 
 
 
 
 panic/issue_2584/final 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 5,000 
 
 
 
 
 
 
 10,000 
 
 
 
 
 
 
 15,000 
 
 
 
 
 
 
 20,000 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 TotalTimeMillis 
 
 
 
 
 Instance